In [1]:
import pandas as pd
import gensim
import nltk
import spacy

In [2]:
oly = pd.read_csv('Olympics-Twitter.csv', encoding = 'utf-8')
# oly.head()

In [3]:
oly.shape

(19714, 41)

In [4]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dsilva2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dsilva2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
sometext = oly['Full Text'].iloc[0]
sometext

"@TeamUSA Literally all of the 2012 Olympics for women's artistic gymnastics"

In [6]:
sometext.lower()

"@teamusa literally all of the 2012 olympics for women's artistic gymnastics"

In [7]:
sometext.upper()

"@TEAMUSA LITERALLY ALL OF THE 2012 OLYMPICS FOR WOMEN'S ARTISTIC GYMNASTICS"

In [8]:
sometext.title()

"@Teamusa Literally All Of The 2012 Olympics For Women'S Artistic Gymnastics"

In [9]:
sometext.split()

['@TeamUSA',
 'Literally',
 'all',
 'of',
 'the',
 '2012',
 'Olympics',
 'for',
 "women's",
 'artistic',
 'gymnastics']

In [10]:
sometext.strip()

"@TeamUSA Literally all of the 2012 Olympics for women's artistic gymnastics"

In [11]:
test = "\n\n\nthis thing"
print(test)




this thing


In [12]:
test.strip()

'this thing'

In [13]:
! python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------- ------------------------------ 2.9/12.8 MB 13.9 MB/s eta 0:00:01
     ------------------ --------------------- 5.8/12.8 MB 14.1 MB/s eta 0:00:01
     --------------------------- ------------ 8.9/12.8 MB 14.6 MB/s eta 0:00:01
     ----------------------------------- --- 11.8/12.8 MB 14.2 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 13.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
othertext = ['my', 'dogs', 'ate', 'carrots', 'yesterday']

nltk.stem.wordnet.WordNetLemmatizer().lemmatize(othertext[3])

'carrot'

In [15]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(sometext)
type(doc)

spacy.tokens.doc.Doc

In [16]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

@TeamUSA PROPN ROOT
Literally ADV advmod
all PRON npadvmod
of ADP prep
the DET det
2012 NUM nummod
Olympics PROPN pobj
for ADP prep
women NOUN poss
's PART case
artistic ADJ amod
gymnastics NOUN pobj


In [17]:
print(nlp('dogs')[0].orth_.isspace(),
      nlp('dogs')[0].like_url,
      nlp('dogs')[0].orth_.startswith('d'),
      nlp('dogs')[0].lower_,
      nlp('dogs')[0].orth_[1:]
)

False False True dogs ogs


In [18]:
nlp('my dogs')[1].like_url

False

In [19]:
def tokenize(text):
    lda_tokens = []
    tokens = nlp(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append(token.orth_[1:])
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [20]:
en_stop = nltk.corpus.stopwords.words('english')
get_lemma = nltk.stem.wordnet.WordNetLemmatizer().lemmatize

In [21]:
def prepare_text(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2] # manipulate this
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [22]:
twt_data = []
for text in oly['Full Text'][:100]:
    twt_data.append(prepare_text(text))

In [23]:
for i in twt_data[0:5]:
    print(i)

['TeamUSA', 'literally', '2012', 'olympics', 'woman', 'artistic', 'gymnastics']
['LanderVBALL', 'reesebatesvb', 'vbcoch', 'USAVBeach', 'usavolleyball', 'MadiReed42', 'nell_copeland', 'omg', 'going', 'try', 'need', 'find', 'another', 'ball']
['blunt', 'started', 'woman', 'basketball', 'country', 'gave', 'notoriety', 'deserved', '-@s10bird', 'close', 'womenshistorymonth', 'celebrating', 'team', 'started', 'dynasty', '1996', 'woman', 'usabasketball', 'URL']
['DetroitHandball', 'TeamHandball', 'USATH', 'USARugby', 'still', 'love']
['beat', 'record', 'new', 'one', 'USAVBeach', 'usavolleyball', 'FIVBBeach', 'FIVBVolleyball', 'OleMissVB', 'NEVBProbs', 'FAUVolleyball', 'FGCU_VB', 'CreightonVB', 'StanfordWVB', 'KUVolleyball', 'URL']


In [24]:
for i in oly['Full Text'][0:5]:
    print(i)

@TeamUSA Literally all of the 2012 Olympics for women's artistic gymnastics
@LanderVBALL @reesebatesvb @vbcoch @USAVBeach @usavolleyball @MadiReed42 @nell_copeland Omg I am going to try this! Now I just need to find another ball🤔
"To be blunt, it started women's basketball in this country. It gave it the notoriety it deserved." -@s10bird We close #WomensHistoryMonth by celebrating the team that started a DYNASTY: The 1996 women of @usabasketball. https://t.co/TI5wWOtjtP
@DetroitHandball @TeamHandball @USATH @USARugby I still love u
I beat my own record! The new one is 52!!! @USAVBeach @usavolleyball @FIVBBeach @FIVBVolleyball @OleMissVB @NEVBProbs @FAUVolleyball @FGCU_VB @CreightonVB @StanfordWVB @KUVolleyball https://t.co/rXutjelgxs


In [25]:
dictionary = gensim.corpora.Dictionary(twt_data)
corpus = [dictionary.doc2bow(text) for text in twt_data]

In [26]:
for i in corpus[0:10]:
    print(i)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]
[(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]
[(6, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1)]
[(37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)]
[(9, 1), (19, 1), (23, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1)]
[(37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1)]
[(54, 1), (65, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1)]
[(1, 1), (23, 1), (86, 1), (87, 1)]
[(88, 1), (89, 1), (90, 2), (91, 1)]
[(1, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1)]


In [27]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                           num_topics = 10,
                                           id2word=dictionary,
                                           passes=15)
ldamodel.print_topics(num_words = 5)

[(0,
  '0.020*"URL" + 0.020*"lol" + 0.013*"kid" + 0.013*"team" + 0.007*"Tokyo2020"'),
 (1,
  '0.070*"USWNT" + 0.036*"alexmorgan13" + 0.036*"samanthabarry" + 0.036*"glamourmag" + 0.027*"FIFAcom"'),
 (2,
  '0.023*"APSWrestle" + 0.016*"URL" + 0.016*"USAWrestling" + 0.016*"calUSAwrestling" + 0.016*"DollamurWrestle"'),
 (3,
  '0.033*"TeamUSA" + 0.018*"URL" + 0.017*"..." + 0.017*"Tokyo2020" + 0.015*"FIFAWWC"'),
 (4,
  '0.021*"good" + 0.014*"USAWrestling" + 0.014*"TeamUSA" + 0.014*"wait" + 0.014*"woman"'),
 (5,
  '0.017*"USMNT" + 0.017*"URL" + 0.017*"RobStoneONFOX" + 0.017*"day" + 0.017*"USWNT"'),
 (6,
  '0.029*"TeamUSA" + 0.024*"URL" + 0.012*"USWNT" + 0.012*"wsl" + 0.012*"look"'),
 (7,
  '0.031*"URL" + 0.023*"USARugby" + 0.023*"USATH" + 0.023*"TeamHandball" + 0.019*"TeamUSA"'),
 (8,
  '0.107*"TeamUSA" + 0.043*"miracle" + 0.036*"1980" + 0.036*"ice" + 0.029*"usa"'),
 (9,
  '0.054*"TeamUSA" + 0.022*"jesse" + 0.022*"owen" + 0.022*"azi" + 0.022*"TeamShuster"')]

In [28]:
ldamodel.get_document_topics(corpus[4])

[(0, 0.9470554)]

In [29]:
oly['Full Text'][4]

'I beat my own record! The new one is 52!!! @USAVBeach @usavolleyball @FIVBBeach @FIVBVolleyball @OleMissVB @NEVBProbs @FAUVolleyball @FGCU_VB @CreightonVB @StanfordWVB @KUVolleyball https://t.co/rXutjelgxs'

In [30]:
twt_data[4]

['beat',
 'record',
 'new',
 'one',
 'USAVBeach',
 'usavolleyball',
 'FIVBBeach',
 'FIVBVolleyball',
 'OleMissVB',
 'NEVBProbs',
 'FAUVolleyball',
 'FGCU_VB',
 'CreightonVB',
 'StanfordWVB',
 'KUVolleyball',
 'URL']